In [ ]:
import os
import json
from collections import defaultdict
import pandas as pd

# Pandas 출력 옵션 설정
pd.set_option('display.max_columns', None)  # 모든 열을 출력
pd.set_option('display.expand_frame_repr', False)  # 가로로 긴 데이터프레임 출력

# JSON 파일에서 성별, 스타일, 선호 여부, 이미지 파일명을 추출하는 함수
def collect_preference_data(image_ids, label_dir):
    preference_data = defaultdict(lambda: {'선호': [], '비선호': []})

    for filename in os.listdir(label_dir):
        if filename.endswith('.json'):
            image_id = extract_image_id(filename)
            if image_id in image_ids:  # 이미지ID가 유효한 경우에만 처리
                json_path = os.path.join(label_dir, filename)
                with open(json_path, 'r', encoding='utf-8') as file:
                    data = json.load(file)
                    respondent_id = data['user']['R_id']
                    style = data['item']['style']
                    img_filename = data['item']['imgName']  # 이미지 파일명
                    preference = data['item']['survey']['Q5']  # 1: 비선호, 2: 선호
                    
                    if preference == 1:
                        preference_data[respondent_id]['비선호'].append(img_filename)
                    elif preference == 2:
                        preference_data[respondent_id]['선호'].append(img_filename)
    
    return preference_data


training_image_ids = collect_image_ids(training_image_dir)
validation_image_ids = collect_image_ids(validation_image_dir)

training_preference_data = collect_preference_data(training_image_ids, training_label_dir)
validation_preference_data = collect_preference_data(validation_image_ids, validation_label_dir)

# Training과 Validation에서 공통된 응답자ID 찾기
common_respondents = set(training_preference_data.keys()) & set(validation_preference_data.keys())

# 응답자별 총 응답수 기준으로 상위 100명 선택
respondent_response_count = {
    respondent_id: len(training_preference_data[respondent_id]['선호']) + len(training_preference_data[respondent_id]['비선호']) +
                   len(validation_preference_data[respondent_id]['선호']) + len(validation_preference_data[respondent_id]['비선호'])
    for respondent_id in common_respondents
}

# 총 응답수 기준 상위 100명의 응답자 선택
top_100_respondents = sorted(respondent_response_count, key=respondent_response_count.get, reverse=True)[:100]

def generate_combined_preference_table_separated(training_data, validation_data, top_respondents):
    rows = []
    for respondent_id in top_respondents:
        training_preferences = training_data.get(respondent_id, {'선호': [], '비선호': []})
        validation_preferences = validation_data.get(respondent_id, {'선호': [], '비선호': []})
        
        rows.append([
            respondent_id,
            ', '.join(training_preferences['선호']),   # Training 스타일 선호 (이미지 파일명)
            ', '.join(training_preferences['비선호']), # Training 스타일 비선호 (이미지 파일명)
            ', '.join(validation_preferences['선호']), # Validation 스타일 선호 (이미지 파일명)
            ', '.join(validation_preferences['비선호']) # Validation 스타일 비선호 (이미지 파일명)
        ])
    
    df = pd.DataFrame(rows, columns=['응답자 ID', 'Training 스타일 선호', 'Training 스타일 비선호', 'Validation 스타일 선호', 'Validation 스타일 비선호'])
    
    # Markdown 형식으로 출력
    print(df.to_markdown(index=False))  # 인덱스 제외하고 출력
    return df

# 표 작성
top_100_preference_df = generate_combined_preference_table_separated(training_preference_data, validation_preference_data, top_100_respondents)